In [ ]:
import sys
sys.path.insert(0, "..")
from src.config import PROJECT_ROOT

# -*- coding: utf-8 -*-
# Requirements: pandas, numpy, scipy, seaborn, matplotlib

import pandas as pd
import numpy as np
import glob, re
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# -----------------------------------------------------------------
# 1. 설정: 파일이 있는 기본 경로
# -----------------------------------------------------------------
# ‼️‼️ 본인 환경에 맞게 이 경로를 확인해주세요. ‼️‼️
BASE_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20")
)


# -----------------------------------------------------------------
# (공용) 도메인 분류 함수 (먼저 정의)
# -----------------------------------------------------------------
def infer_domain(feat: str) -> str:
    t = str(feat).lower().strip()

    # -----------------------------------------------------------------
    # 1. SDoMH (Social Determinants of Mental Health) - LLM 관련 변수
    # -----------------------------------------------------------------
    # LLM 변수들은 모두 SDoMH 도메인으로 분류하되,
    # 식별을 위해 (LLM) 접미사 혹은 원본 키워드를 체크합니다.
    SDOMH_KEYWORDS = [
        "social function",
        "social isolation",
        "religious",
        "violence",
        "impulsivity",
        "abuse",
        "neglect",
        "divorce",
        "family member",
        "family support",
        "psychotic",
        "halluc",
        "delusion",
        "paranoia",
        "interpersonal",
        "suicide exposure",
        "alcohol use",
        "victimization",
    ]

    # 변수명에 (LLM)이 명시되어 있거나 SDoMH 관련 핵심 키워드가 포함된 경우
    if "(llm)" in t or any(k in t for k in SDOMH_KEYWORDS):
        # 특정 그룹으로 더 세분화하고 싶다면 여기서 return 값을 조정할 수 있습니다.
        # 예: return "SDoMH (Social)" 등
        return "SDoMH"

    # -----------------------------------------------------------------
    # 2. Utilization (의료 이용)
    # -----------------------------------------------------------------
    if any(
        k in t
        for k in [
            "er visit",
            "≥2 er",
            ">=2 er",
            "≥3 er",
            ">=3 er",
            "admission",
            "admit",
            "er_more_two",
        ]
    ):
        return "Utilization"

    # -----------------------------------------------------------------
    # 3. Lab (검사 수치)
    # -----------------------------------------------------------------
    if any(
        k in t
        for k in [
            "bl",
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "lymphocyte",
            "monocyte",
            "acl",
            "platelet",
            "creatinine",
            "ast",
            "alt",
            "alp",
            "bun",
            "calcium",
            "phospho",
        ]
    ):
        return "Lab"

    # -----------------------------------------------------------------
    # 4. Psychometric (심리 검사 지표)
    # -----------------------------------------------------------------
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trin",
            "trinf",
            "psychometric",
            "audit",
            "asrs",
            "bai",
            "bhs",
            "ham",
            "hcl",
            "perceptual",
            "appq",
            "bprs",
            "gaf",
            "suicidalidea",
            "suicidalplan",
            "suicidalattempt",
        ]
    ):
        return "Psychometric"

    # -----------------------------------------------------------------
    # 5. Treatment (약물 및 치료)
    # -----------------------------------------------------------------
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "medic",
            "benzodiazepine",
            "risper",
            "valpro",
            "ssri",
            "snri",
            "quetiapine",
            "aripip",
            "olanzapine",
            "divalproex",
            "aripiprazole",
            "lithium",
            "quetiapine",
        ]
    ):
        return "Treatment"

    # -----------------------------------------------------------------
    # 6. Sleep/Appetite (생체 리듬)
    # -----------------------------------------------------------------
    if "sleep" in t or "appetite" in t or "weight" in t:
        return "Biological"

    # -----------------------------------------------------------------
    # 7. Demographic/Psychosocial (인구학적 정보 및 기존 진단명)
    # -----------------------------------------------------------------
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "marry",
            "age",
            "education",
            "edu",
            "job",
            "occupation",
            "income",
            "smoking",
            "smoke",
            "drinking",
            "drink",
            "psychiatric family",
            "psy_family",
            "hospitalization",
            "length of stay",
            "stay_day",
            "bipolar",
            "depression",
            "schizophrenia",
            "anxiety",
            "somatic symptom",
            "trauma_stressor",
        ]
    ):
        return "Demographic/Psychosocial"

    return "Other"


# -----------------------------------------------------------------
# 2. 데이터 로딩
# -----------------------------------------------------------------
print("파일 로드를 시작합니다...")

# (A) Load LONG files
long_files = [
    ("30d", "label_30d__SHAP_long_Test.csv"),
    ("60d", "label_60d__SHAP_long_Test.csv"),
    ("90d", "label_90d__SHAP_long_Test.csv"),
    ("180d", "label_180d__SHAP_long_Test.csv"),
    ("365d", "label_365d__SHAP_long_Test.csv"),
]
long_dfs = []
for tp, fname in long_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        long_dfs.append(df)
    else:
        print(f"Warning: {file_path} 파일을 찾을 수 없습니다.")
if not long_dfs:
    raise ValueError("Long 파일을 하나도 찾지 못했습니다.")
full_long = pd.concat(long_dfs, ignore_index=True)

# (B) Load IMPORTANCE files
imp_files = [
    ("30d", "label_30d__SHAP_feature_importance_Test.csv"),
    ("60d", "label_60d__SHAP_feature_importance_Test.csv"),
    ("90d", "label_90d__SHAP_feature_importance_Test.csv"),
    ("180d", "label_180d__SHAP_feature_importance_Test.csv"),
    ("365d", "label_365d__SHAP_feature_importance_Test.csv"),
]
imp_dfs = []
for tp, fname in imp_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        imp_dfs.append(df)
    else:
        print(f"Warning: Importance 파일을 찾을 수 없습니다: {file_path}")
if not imp_dfs:
    raise ValueError("Importance 파일을 하나도 찾지 못했습니다.")
full_imp = pd.concat(imp_dfs, ignore_index=True)

print("파일 로드 완료. 상관관계 분석을 시작합니다...")

# -----------------------------------------------------------------
# 3. 상관관계 계산 및 데이터 가공
# -----------------------------------------------------------------


def calc_correlation(g):
    if g["feature_value"].nunique() <= 1 or g["shap_value"].nunique() <= 1:
        corr = 0
    else:
        corr, _ = spearmanr(g["feature_value"], g["shap_value"])
        if np.isnan(corr):
            corr = 0
    return pd.Series({"corr_direction": corr})


metrics_df = (
    full_long.groupby(["time_point", "feature"]).apply(calc_correlation).reset_index()
)

time_order = ["30d", "60d", "90d", "180d", "365d"]
TOP_N = 20

# (A) 중요도 피벗 (모든 변수) - ANNOTATION (TEXT) 용
imp_pivot = full_imp.pivot(
    index="feature", columns="time_point", values="mean_abs_shap"
).reindex(columns=time_order)

# (B) Top 20 필터링 (어느 시점에서든 Top 20 안에 든 변수 추출)
top_features_union = set()
for tp in time_order:
    if tp in imp_pivot.columns:
        top_in_tp = imp_pivot[tp].fillna(-1).nlargest(TOP_N).index.tolist()
        top_features_union.update(top_in_tp)

# (C) 방향성 피벗 - DATA (COLOR) 용
dir_pivot = metrics_df.pivot(
    index="feature", columns="time_point", values="corr_direction"
).reindex(columns=time_order)

# (D) 필터 적용 및 정렬
# 1. Color Data (Correlation)
dir_top = dir_pivot.loc[list(top_features_union)].copy()
# 2. Annotation Data (Importance)
imp_top_filtered = imp_pivot.loc[list(top_features_union)].copy()

# 3. 정렬 (평균 중요도 기준)
mean_imp_overall = imp_top_filtered.mean(axis=1)
sorted_idx = mean_imp_overall.sort_values(ascending=False).index

dir_top = dir_top.loc[sorted_idx]
imp_top_filtered = imp_top_filtered.loc[sorted_idx]  # Annotation도 동일하게 정렬

print(f"총 {len(dir_top)}개의 변수로 시각화 및 테이블 생성을 준비합니다.")

# -----------------------------------------------------------------
# 4. (수정) 플롯 2개로 분할 (Color=Corr, Text=Importance)
# -----------------------------------------------------------------

# (A) 데이터 분할
midpoint = int(np.ceil(len(dir_top) / 2))
# 1. Color Data
dir_top_part1 = dir_top.iloc[:midpoint]
dir_top_part2 = dir_top.iloc[midpoint:]
# 2. Annotation Data
annot_data_part1 = imp_top_filtered.iloc[:midpoint]
annot_data_part2 = imp_top_filtered.iloc[midpoint:]

print(
    f"플롯을 Part1 ({len(dir_top_part1)}개), Part2 ({len(dir_top_part2)}개)로 분할합니다."
)

# (B) 공통 플롯 설정
font_sizes = {"title": 18, "label": 14, "tick": 12, "annot": 12, "colorbar_label": 14}
cbar_kws_dict = {
    "label": "Spearman Correlation (Feature Value vs. SHAP Value)",
    "shrink": 0.7,
    "ticklocation": "right",
}

# (C) 플롯 1 그리기
plt.figure(figsize=(12, len(dir_top_part1) * 0.5 + 2.5))
sns.heatmap(
    data=dir_top_part1,  # ⬅️ 색상: 상관관계 (dir_top)
    annot=annot_data_part1,  # ⬅️ 숫자: 중요도 (imp_top)
    fmt=".3f",  # ⬅️ 포맷: 중요도 기준 (소수점 3자리)
    cmap="coolwarm",
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=1.2,
    linecolor="gray",
    cbar_kws=cbar_kws_dict,
    annot_kws={"size": font_sizes["annot"], "weight": "bold", "color": "black"},
)
plt.title(
    "SHAP Directionality \n(Color=Direction, Text=Mean|SHAP|)",
    fontsize=font_sizes["title"],
    fontweight="bold",
    pad=25,
)
plt.xlabel("Time Point", fontsize=font_sizes["label"], fontweight="bold")
plt.ylabel("Feature", fontsize=font_sizes["label"], fontweight="bold")
plt.xticks(fontsize=font_sizes["tick"], rotation=0)
plt.yticks(fontsize=font_sizes["tick"])
cbar = plt.gca().collections[0].colorbar
cbar.set_label(
    cbar_kws_dict["label"], fontsize=font_sizes["colorbar_label"], fontweight="bold"
)
cbar.ax.tick_params(labelsize=font_sizes["tick"])
plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.savefig(BASE_DIR / "Analysis1_SHAP_Heatmap_Hybrid_Part1.png", dpi=400)
plt.close()
print(f"저장 완료: {BASE_DIR / 'Analysis1_SHAP_Heatmap_Hybrid_Part1.png'}")

# (D) 플롯 2 그리기
plt.figure(figsize=(12, len(dir_top_part2) * 0.5 + 2.5))
sns.heatmap(
    data=dir_top_part2,  # ⬅️ 색상: 상관관계 (dir_top)
    annot=annot_data_part2,  # ⬅️ 숫자: 중요도 (imp_top)
    fmt=".3f",  # ⬅️ 포맷: 중요도 기준 (소수점 3자리)
    cmap="coolwarm",
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=1.2,
    linecolor="gray",
    cbar_kws=cbar_kws_dict,
    annot_kws={"size": font_sizes["annot"], "weight": "bold", "color": "black"},
)
plt.title(
    "SHAP Directionality \n(Color=Direction, Text=Mean|SHAP|)",
    fontsize=font_sizes["title"],
    fontweight="bold",
    pad=25,
)
plt.xlabel("Time Point", fontsize=font_sizes["label"], fontweight="bold")
plt.ylabel("Feature", fontsize=font_sizes["label"], fontweight="bold")
plt.xticks(fontsize=font_sizes["tick"], rotation=0)
plt.yticks(fontsize=font_sizes["tick"])
cbar = plt.gca().collections[0].colorbar
cbar.set_label(
    cbar_kws_dict["label"], fontsize=font_sizes["colorbar_label"], fontweight="bold"
)
cbar.ax.tick_params(labelsize=font_sizes["tick"])
plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.savefig(BASE_DIR / "Analysis1_SHAP_Heatmap_Hybrid_Part2.png", dpi=400)
plt.close()
print(f"저장 완료: {BASE_DIR / 'Analysis1_SHAP_Heatmap_Hybrid_Part2.png'}")


# -----------------------------------------------------------------
# 5. CSV 파일 저장 (Strongest, Consistency, Domain, Symbol + Importance 통합)
# -----------------------------------------------------------------
print("최종 상세 테이블 생성을 시작합니다...")


# (A) 단순화된 기호 정의 (기준 0.3으로 상향)
def get_corr_symbol_simple(val):
    if pd.isna(val):
        return ""
    if val >= 0.3:
        return "🔺"
    if val <= -0.3:
        return "▼"
    return "• (Neutral)"


dir_symbol_top = dir_top.applymap(get_corr_symbol_simple)
# imp_top_filtered는 3번 섹션에서 이미 dir_top과 동일하게 정렬되었음

# (B) 통합 테이블 생성
combined_table = pd.DataFrame(index=dir_top.index)
for tp in time_order:
    if tp in dir_symbol_top.columns:
        combined_table[f"{tp}_direction"] = dir_symbol_top[tp]
        # [수정] 소수점 4자리 반올림
        combined_table[f"{tp}_SHAP(mean)"] = imp_top_filtered[tp].round(4)

# (C) Strongest window, Domain 추가
strongest_window = imp_top_filtered.idxmax(axis=1)
domains = combined_table.index.map(infer_domain)

# (D) Top 20 Count (n/5) 계산 (컬럼명 변경)
rank_pivot = imp_pivot.rank(axis=0, method="min", ascending=False)
rank_pivot_top = rank_pivot.loc[dir_top.index]
is_top_n = rank_pivot_top <= TOP_N
consistency_count = is_top_n.sum(axis=1)
consistency_str = consistency_count.astype(str) + f"/{len(time_order)}"

# [수정] 컬럼 순서 변경
combined_table.insert(0, "Domain", domains)
combined_table["Strongest Window"] = strongest_window
combined_table["Top 20 Count (n/5)"] = consistency_str

# (F) 파일로 저장
combined_table.to_excel(BASE_DIR / "Analysis1_SHAP_Domain_Symbol_Importance_FINAL.xlsx")
print(
    f"★★★ 최종 상세 테이블 저장 완료: {BASE_DIR / 'Analysis1_SHAP_Domain_Symbol_Importance_FINAL.xlsx'}"
)


# -----------------------------------------------------------------
# 6. Top 20 도메인 비율 요약 테이블
# -----------------------------------------------------------------
print("\nTop 20 도메인 요약 테이블 생성을 시작합니다...")

# (A) 각 시점별 Top 20 변수 추출 및 도메인 계산
top_n_features_by_time = (
    full_imp.groupby("time_point")
    .apply(lambda x: x.nlargest(TOP_N, "mean_abs_shap"))
    .reset_index(drop=True)
)
top_n_features_by_time["Domain"] = top_n_features_by_time["feature"].apply(infer_domain)

# (B) 시점별 도메인 카운트 집계
domain_counts_pivot = (
    top_n_features_by_time.groupby("time_point")["Domain"]
    .value_counts()
    .unstack(level="time_point")
    .fillna(0)
    .astype(int)
)
domain_counts_pivot = domain_counts_pivot.reindex(columns=time_order, fill_value=0)

# (C) 비율(%) 계산
domain_proportions_pivot = domain_counts_pivot / TOP_N * 100

# (D) 최종 포맷팅: "Proportion % (Count)"
formatted_table = pd.DataFrame(
    index=domain_counts_pivot.index, columns=domain_counts_pivot.columns
)
for col in formatted_table.columns:
    counts = domain_counts_pivot[col]
    props = domain_proportions_pivot[col]
    formatted_table[col] = [
        f"{p:.1f}% ({c})" if c > 0 else "-" for c, p in zip(counts, props)
    ]

# (E) 총합(Total) 행 추가
total_counts = domain_counts_pivot.sum()
total_row = [f"100.0% ({c})" for c in total_counts]
formatted_table.loc["Total"] = total_row

# (F) 파일로 저장
domain_table_path = BASE_DIR / "Analysis1_SHAP_Domain_Proportions_Summary.xlsx"
formatted_table.to_excel(domain_table_path)

print(f"★★★ 신규 도메인 비율 요약 테이블 저장 완료: {domain_table_path}")
print("\n모든 작업이 완료되었습니다.")

In [ ]:
# -----------------------------------------------------------------
# 1. 설정: 파일이 있는 기본 경로
# -----------------------------------------------------------------
# ‼️‼️ 본인 환경에 맞게 이 경로를 확인해주세요. ‼️‼️
BASE_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20")
)


# -----------------------------------------------------------------
# (공용) 도메인 분류 함수 (먼저 정의)
# -----------------------------------------------------------------
def infer_domain(feat: str) -> str:
    t = str(feat).lower().strip()
    # ... (제공해주신 도메인 분류 함수 내용 전체) ...
    LLM_ALIASES = {
        "Aggression (LLM)": ["aggression", "aggressive"],
        "Interpersonal Conflict (LLM)": [
            "interpersonal conflict",
            "relationship conflict",
            "family conflict",
            "interpersonal",
        ],
        "Impaired Social Functioning (LLM)": [
            "impaired social functioning",
            "social function impairment",
            "social functioning impairment",
        ],
        "Alcohol Use Issues (LLM)": [
            "alcohol use issues",
            "alcohol use",
            "alcohol",
            "drinking",
        ],
        "Hallucinations / Delusions / Paranoia (LLM)": [
            "hallucinations_delusions_paranoia",
            "hallucinations",
            "delusions",
            "paranoia",
            "halluc/delusion/paranoia",
        ],
        "Lack of Support (LLM)": [
            "lack family social support",
            "lack of support",
            "poor support",
            "no support",
            "social support deficit",
        ],
        "Family Loss (LLM)": [
            "loss_of_family",
            "family loss",
            "loss / grief",
            "bereavement",
        ],
        "Abuse / Sexual Victimization (LLM)": [
            "abuse_and_sexual_victimization",
            "abuse",
            "victimization",
            "sexual",
            "violence",
            "victim",
        ],
    }
    for dom, keys in LLM_ALIASES.items():
        if any(k in t for k in keys) and "llm" in t:
            return dom
    if any(
        k in t
        for k in [
            "er visit",
            "≥2 er",
            ">=2 er",
            "≥3 er",
            ">=3 er",
            "admission",
            "admit",
        ]
    ):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl",
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "lymphocyte",
            "monocyte",
            "acl",
            "platelet",
            "creatinine",
            "ast",
            "alt",
            "alp",
            "bun",
            "calcium",
            "phospho",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trin",
            "trinf",
            "psychometric",
            "audit",
            "asrs",
            "bai",
            "bhs",
            "ham",
            "hcl",
            "perceptual",
            "appq",
            "bprs",
            "gaf",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "medic",
            "benzodiazepine",
            "risper",
            "valpro",
            "ssri",
            "snri",
            "quetiapine",
            "aripip",
            "olanzapine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "occupation",
            "income",
            "smoking",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
            "length of stay",
            "other psychotic",
            "somatic symptom disorder",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# -----------------------------------------------------------------
# 2. 데이터 로딩
# -----------------------------------------------------------------
print("파일 로드를 시작합니다...")

# (A) Load LONG files
long_files = [
    ("30d", "label_30d__SHAP_long_Test.csv"),
    ("60d", "label_60d__SHAP_long_Test.csv"),
    ("90d", "label_90d__SHAP_long_Test.csv"),
    ("180d", "label_180d__SHAP_long_Test.csv"),
    ("365d", "label_365d__SHAP_long_Test.csv"),
]
long_dfs = []
for tp, fname in long_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        long_dfs.append(df)
    else:
        print(f"Warning: {file_path} 파일을 찾을 수 없습니다.")
if not long_dfs:
    raise ValueError("Long 파일을 하나도 찾지 못했습니다.")
full_long = pd.concat(long_dfs, ignore_index=True)

# (B) Load IMPORTANCE files
imp_files = [
    ("30d", "label_30d__SHAP_feature_importance_Test.csv"),
    ("60d", "label_60d__SHAP_feature_importance_Test.csv"),
    ("90d", "label_90d__SHAP_feature_importance_Test.csv"),
    ("180d", "label_180d__SHAP_feature_importance_Test.csv"),
    ("365d", "label_365d__SHAP_feature_importance_Test.csv"),
]
imp_dfs = []
for tp, fname in imp_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        imp_dfs.append(df)
    else:
        print(f"Warning: Importance 파일을 찾을 수 없습니다: {file_path}")
if not imp_dfs:
    raise ValueError("Importance 파일을 하나도 찾지 못했습니다.")
full_imp = pd.concat(imp_dfs, ignore_index=True)

print("파일 로드 완료. 상관관계 분석을 시작합니다...")

# -----------------------------------------------------------------
# 3. 상관관계 계산 및 데이터 가공
# -----------------------------------------------------------------


# -----------------------------------------------------------------
# 3. 상관관계 계산 및 데이터 가공 (사용자 지정 변수 필터링 적용)
# -----------------------------------------------------------------


# (A) 상관관계 계산 함수 (기존 유지)
def calc_correlation(g):
    if g["feature_value"].nunique() <= 1 or g["shap_value"].nunique() <= 1:
        corr = 0
    else:
        corr, _ = spearmanr(g["feature_value"], g["shap_value"])
        if np.isnan(corr):
            corr = 0
    return pd.Series({"corr_direction": corr})


print("상관관계 계산 중...")
metrics_df = (
    full_long.groupby(["time_point", "feature"]).apply(calc_correlation).reset_index()
)

time_order = ["30d", "60d", "90d", "180d", "365d"]

# (B) 전체 피벗 테이블 생성
# 1. Annotation Data (Importance)
imp_pivot = full_imp.pivot(
    index="feature", columns="time_point", values="mean_abs_shap"
).reindex(columns=time_order)

# 2. Color Data (Correlation)
dir_pivot = metrics_df.pivot(
    index="feature", columns="time_point", values="corr_direction"
).reindex(columns=time_order)


# -----------------------------------------------------------------
# [수정] Top N 자동 추출 대신 사용자 지정 리스트 사용
# -----------------------------------------------------------------

# ‼️‼️ 여기에 시각화하고 싶은 변수 이름을 정확히 입력하세요 ‼️‼️
TARGET_FEATURES = [
    "≥2 ER Visits",
    "≥3 Admissions",
    "ANC (BL201815)",
    "CRP (C-Reactive Protein) (BL3140)",
    "Glucose, Fasting (BL3118)",
    "Abuse/Victimization (LLM)",
    "Halluc/Delusion/Paranoia (LLM)",
    "Lack of Support (LLM)",
    "Interpersonal Conflict (LLM)",
    # ... 필요한 변수들을 계속 추가하세요
]

# (C) 데이터 필터링 (지정한 변수만 남기기)
# 데이터에 실제로 존재하는 변수인지 확인 (오타 방지)
available_features = imp_pivot.index.tolist()
selected_features = [f for f in TARGET_FEATURES if f in available_features]
missing_features = set(TARGET_FEATURES) - set(selected_features)

if missing_features:
    print(
        f"\n[Warning] 다음 변수들은 데이터셋에 존재하지 않아 제외되었습니다:\n{missing_features}\n"
    )

if not selected_features:
    raise ValueError("선택된 변수가 하나도 없습니다. 변수 이름을 확인해주세요.")

# 선택된 변수로 데이터프레임 필터링
dir_top = dir_pivot.loc[selected_features].copy()
imp_top_filtered = imp_pivot.loc[selected_features].copy()

# (D) 정렬 (옵션)
# 1. 사용자가 적은 리스트 순서 그대로 보고 싶다면: 아래 3줄 주석 처리
# 2. 중요도(Mean SHAP) 순으로 정렬하고 싶다면: 아래 코드 유지
mean_imp_overall = imp_top_filtered.mean(axis=1)
sorted_idx = mean_imp_overall.sort_values(ascending=False).index

dir_top = dir_top.loc[sorted_idx]
imp_top_filtered = imp_top_filtered.loc[sorted_idx]

print(f"총 {len(dir_top)}개의 지정된 변수로 시각화 및 테이블 생성을 준비합니다.")

# -----------------------------------------------------------------
# 4. (수정) 통합 플롯 그리기 (하나의 이미지로 저장)
# -----------------------------------------------------------------

print(f"총 {len(dir_top)}개의 변수를 하나의 히트맵으로 시각화합니다.")

# (A) 플롯 설정
# 변수 개수에 따라 높이 자동 조절 (변수당 0.5인치 + 기본 여백 3인치)
# 변수가 많아져도 셀이 찌그러지지 않도록 높이를 동적으로 할당합니다.
fig_height = len(dir_top) * 0.5 + 3
plt.figure(figsize=(12, fig_height))

font_sizes = {"title": 18, "label": 14, "tick": 12, "annot": 12, "colorbar_label": 14}

# 컬러바 설정
cbar_kws_dict = {
    "label": "Spearman Correlation (Feature Value vs. SHAP Value)",
    "shrink": 0.6,  # 전체 플롯 높이에 맞춰 컬러바 길이 비율 조정
    "ticklocation": "right",
}

# (B) 히트맵 그리기 (분할 없이 dir_top 전체 사용)
sns.heatmap(
    data=dir_top,  # ⬅️ 색상: 상관관계 (전체)
    annot=imp_top_filtered,  # ⬅️ 숫자: 중요도 (전체)
    fmt=".3f",
    cmap="coolwarm",
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=1.2,
    linecolor="gray",
    cbar_kws=cbar_kws_dict,
    annot_kws={"size": font_sizes["annot"], "weight": "bold", "color": "black"},
)

# (C) 레이블 및 타이틀 설정
plt.title(
    "SHAP Directionality (Selected Features)\n(Color=Direction, Text=Mean|SHAP|)",
    fontsize=font_sizes["title"],
    fontweight="bold",
    pad=25,
)
plt.xlabel("Time Point", fontsize=font_sizes["label"], fontweight="bold")
plt.ylabel("Feature", fontsize=font_sizes["label"], fontweight="bold")
plt.xticks(fontsize=font_sizes["tick"], rotation=0)
plt.yticks(fontsize=font_sizes["tick"])

# (D) 컬러바 폰트 및 위치 조정
cbar = plt.gca().collections[0].colorbar
cbar.set_label(
    cbar_kws_dict["label"], fontsize=font_sizes["colorbar_label"], fontweight="bold"
)
cbar.ax.tick_params(labelsize=font_sizes["tick"])

# (E) 저장
plt.tight_layout(rect=[0, 0, 1, 0.96])  # 타이틀 공간 확보
save_path = BASE_DIR / "Analysis1_SHAP_Heatmap_All_Selected.png"
plt.savefig(save_path, dpi=400)
plt.close()

print(f"저장 완료: {save_path}")

# # -----------------------------------------------------------------
# # 5. CSV 파일 저장 (Strongest, Consistency, Domain, Symbol + Importance 통합)
# # -----------------------------------------------------------------
# print("최종 상세 테이블 생성을 시작합니다...")


# # (A) 단순화된 기호 정의 (기준 0.3으로 상향)
# def get_corr_symbol_simple(val):
#     if pd.isna(val):
#         return ""
#     if val >= 0.3:
#         return "🔺"
#     if val <= -0.3:
#         return "▼"
#     return "• (Neutral)"


# dir_symbol_top = dir_top.applymap(get_corr_symbol_simple)
# # imp_top_filtered는 3번 섹션에서 이미 dir_top과 동일하게 정렬되었음

# # (B) 통합 테이블 생성
# combined_table = pd.DataFrame(index=dir_top.index)
# for tp in time_order:
#     if tp in dir_symbol_top.columns:
#         combined_table[f"{tp}_direction"] = dir_symbol_top[tp]
#         # [수정] 소수점 4자리 반올림
#         combined_table[f"{tp}_SHAP(mean)"] = imp_top_filtered[tp].round(4)

# # (C) Strongest window, Domain 추가
# strongest_window = imp_top_filtered.idxmax(axis=1)
# domains = combined_table.index.map(infer_domain)

# # (D) Top 20 Count (n/5) 계산 (컬럼명 변경)
# rank_pivot = imp_pivot.rank(axis=0, method="min", ascending=False)
# rank_pivot_top = rank_pivot.loc[dir_top.index]
# is_top_n = rank_pivot_top <= TOP_N
# consistency_count = is_top_n.sum(axis=1)
# consistency_str = consistency_count.astype(str) + f"/{len(time_order)}"

# # [수정] 컬럼 순서 변경
# combined_table.insert(0, "Domain", domains)
# combined_table["Strongest Window"] = strongest_window
# combined_table["Top 20 Count (n/5)"] = consistency_str

# # (F) 파일로 저장
# combined_table.to_excel(BASE_DIR / "short_fig/Analysis1_SHAP_Domain_Symbol_Importance_FINAL_Summary.xlsx")
# print(
#     f"★★★ 최종 상세 테이블 저장 완료: {BASE_DIR / 'short_fig/Analysis1_SHAP_Domain_Symbol_Importance_FINAL_Summary.xlsx'}"
# )


# # -----------------------------------------------------------------
# # 6. Top 20 도메인 비율 요약 테이블
# # -----------------------------------------------------------------
# print("\nTop 20 도메인 요약 테이블 생성을 시작합니다...")

# # (A) 각 시점별 Top 20 변수 추출 및 도메인 계산
# top_n_features_by_time = (
#     full_imp.groupby("time_point")
#     .apply(lambda x: x.nlargest(TOP_N, "mean_abs_shap"))
#     .reset_index(drop=True)
# )
# top_n_features_by_time["Domain"] = top_n_features_by_time["feature"].apply(infer_domain)

# # (B) 시점별 도메인 카운트 집계
# domain_counts_pivot = (
#     top_n_features_by_time.groupby("time_point")["Domain"]
#     .value_counts()
#     .unstack(level="time_point")
#     .fillna(0)
#     .astype(int)
# )
# domain_counts_pivot = domain_counts_pivot.reindex(columns=time_order, fill_value=0)

# # (C) 비율(%) 계산
# domain_proportions_pivot = domain_counts_pivot / TOP_N * 100

# # (D) 최종 포맷팅: "Proportion % (Count)"
# formatted_table = pd.DataFrame(
#     index=domain_counts_pivot.index, columns=domain_counts_pivot.columns
# )
# for col in formatted_table.columns:
#     counts = domain_counts_pivot[col]
#     props = domain_proportions_pivot[col]
#     formatted_table[col] = [
#         f"{p:.1f}% ({c})" if c > 0 else "-" for c, p in zip(counts, props)
#     ]

# # (E) 총합(Total) 행 추가
# total_counts = domain_counts_pivot.sum()
# total_row = [f"100.0% ({c})" for c in total_counts]
# formatted_table.loc["Total"] = total_row

# # (F) 파일로 저장
# domain_table_path = BASE_DIR / "Analysis1_SHAP_Domain_Proportions_Summary.xlsx"
# formatted_table.to_excel(domain_table_path)

# print(f"★★★ 신규 도메인 비율 요약 테이블 저장 완료: {domain_table_path}")
# print("\n모든 작업이 완료되었습니다.")

In [ ]:
# -----------------------------------------------------------------
# 1. 설정: 파일이 있는 기본 경로
# -----------------------------------------------------------------
# ‼️‼️ 본인 환경에 맞게 이 경로를 확인해주세요. ‼️‼️
BASE_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20")
)


# -----------------------------------------------------------------
# (공용) 도메인 분류 함수 (먼저 정의)
# -----------------------------------------------------------------
def infer_domain(feat: str) -> str:
    t = str(feat).lower().strip()
    # ... (제공해주신 도메인 분류 함수 내용 전체) ...
    LLM_ALIASES = {
        "Aggression (LLM)": ["aggression", "aggressive"],
        "Interpersonal Conflict (LLM)": [
            "interpersonal conflict",
            "relationship conflict",
            "family conflict",
            "interpersonal",
        ],
        "Impaired Social Functioning (LLM)": [
            "impaired social functioning",
            "social function impairment",
            "social functioning impairment",
        ],
        "Alcohol Use Issues (LLM)": [
            "alcohol use issues",
            "alcohol use",
            "alcohol",
            "drinking",
        ],
        "Hallucinations / Delusions / Paranoia (LLM)": [
            "hallucinations_delusions_paranoia",
            "hallucinations",
            "delusions",
            "paranoia",
            "halluc/delusion/paranoia",
        ],
        "Lack of Support (LLM)": [
            "lack family social support",
            "lack of support",
            "poor support",
            "no support",
            "social support deficit",
        ],
        "Family Loss (LLM)": [
            "loss_of_family",
            "family loss",
            "loss / grief",
            "bereavement",
        ],
        "Abuse / Sexual Victimization (LLM)": [
            "abuse_and_sexual_victimization",
            "abuse",
            "victimization",
            "sexual",
            "violence",
            "victim",
        ],
    }
    for dom, keys in LLM_ALIASES.items():
        if any(k in t for k in keys) and "llm" in t:
            return dom
    if any(
        k in t
        for k in [
            "er visit",
            "≥2 er",
            ">=2 er",
            "≥3 er",
            ">=3 er",
            "admission",
            "admit",
        ]
    ):
        return "Utilization"
    if any(
        k in t
        for k in [
            "bl",
            "bl3",
            "serum",
            "cholesterol",
            "hdl",
            "ldl",
            "triglyceride",
            "crp",
            "anc",
            "sodium",
            "na",
            "potassium",
            "hematocrit",
            "mchc",
            "eosinophil",
            "basophil",
            "lymphocyte",
            "monocyte",
            "acl",
            "platelet",
            "creatinine",
            "ast",
            "alt",
            "alp",
            "bun",
            "calcium",
            "phospho",
        ]
    ):
        return "Lab"
    if any(
        k in t
        for k in [
            "mmpi",
            "hy",
            "mf",
            "vrin",
            "trin",
            "trinf",
            "psychometric",
            "audit",
            "asrs",
            "bai",
            "bhs",
            "ham",
            "hcl",
            "perceptual",
            "appq",
            "bprs",
            "gaf",
        ]
    ):
        return "Psychometric"
    if any(
        k in t
        for k in [
            "olanz",
            "lith",
            "bzd",
            "quetiap",
            "antipsych",
            "medic",
            "benzodiazepine",
            "risper",
            "valpro",
            "ssri",
            "snri",
            "quetiapine",
            "aripip",
            "olanzapine",
        ]
    ):
        return "Treatment"
    if "sleep" in t:
        return "Sleep"
    if any(
        k in t
        for k in [
            "sex",
            "marital",
            "age",
            "education",
            "job",
            "occupation",
            "income",
            "smoking",
            "drinking",
            "psychiatric family history",
            "hospitalization period",
            "length of stay",
            "other psychotic",
            "somatic symptom disorder",
        ]
    ):
        return "Demographic/Psychosocial"
    return "Other"


# -----------------------------------------------------------------
# 2. 데이터 로딩
# -----------------------------------------------------------------
print("파일 로드를 시작합니다...")

# (A) Load LONG files
long_files = [
    ("30d", "label_30d__SHAP_long_Test.csv"),
    ("60d", "label_60d__SHAP_long_Test.csv"),
    ("90d", "label_90d__SHAP_long_Test.csv"),
    ("180d", "label_180d__SHAP_long_Test.csv"),
    ("365d", "label_365d__SHAP_long_Test.csv"),
]
long_dfs = []
for tp, fname in long_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        long_dfs.append(df)
    else:
        print(f"Warning: {file_path} 파일을 찾을 수 없습니다.")
if not long_dfs:
    raise ValueError("Long 파일을 하나도 찾지 못했습니다.")
full_long = pd.concat(long_dfs, ignore_index=True)

# (B) Load IMPORTANCE files
imp_files = [
    ("30d", "label_30d__SHAP_feature_importance_Test.csv"),
    ("60d", "label_60d__SHAP_feature_importance_Test.csv"),
    ("90d", "label_90d__SHAP_feature_importance_Test.csv"),
    ("180d", "label_180d__SHAP_feature_importance_Test.csv"),
    ("365d", "label_365d__SHAP_feature_importance_Test.csv"),
]
imp_dfs = []
for tp, fname in imp_files:
    file_path = BASE_DIR / fname
    if file_path.exists():
        df = pd.read_csv(file_path)
        df["time_point"] = tp
        imp_dfs.append(df)
    else:
        print(f"Warning: Importance 파일을 찾을 수 없습니다: {file_path}")
if not imp_dfs:
    raise ValueError("Importance 파일을 하나도 찾지 못했습니다.")
full_imp = pd.concat(imp_dfs, ignore_index=True)

print("파일 로드 완료. 상관관계 분석을 시작합니다...")

# -----------------------------------------------------------------
# 3. 상관관계 계산 및 데이터 가공 (사용자 지정 변수 & 순서 유지)
# -----------------------------------------------------------------


# (A) 상관관계 계산 함수 (기존 유지)
def calc_correlation(g):
    if g["feature_value"].nunique() <= 1 or g["shap_value"].nunique() <= 1:
        corr = 0
    else:
        corr, _ = spearmanr(g["feature_value"], g["shap_value"])
        if np.isnan(corr):
            corr = 0
    return pd.Series({"corr_direction": corr})


print("상관관계 계산 중...")
metrics_df = (
    full_long.groupby(["time_point", "feature"]).apply(calc_correlation).reset_index()
)

time_order = ["30d", "60d", "90d", "180d", "365d"]

# (B) 전체 피벗 테이블 생성
# 1. Annotation Data (Importance)
imp_pivot = full_imp.pivot(
    index="feature", columns="time_point", values="mean_abs_shap"
).reindex(columns=time_order)

# 2. Color Data (Correlation)
dir_pivot = metrics_df.pivot(
    index="feature", columns="time_point", values="corr_direction"
).reindex(columns=time_order)


# -----------------------------------------------------------------
# [수정] Top N 자동 추출 대신 사용자 지정 리스트 사용
# -----------------------------------------------------------------

# ‼️‼️ 여기에 시각화하고 싶은 변수 이름을 정확히 입력하세요 ‼️‼️
TARGET_FEATURES = [
    "≥2 ER Visits",
    "≥3 Admissions",
    "ANC (BL201815)",
    "CRP (C-Reactive Protein) (BL3140)",
    "Glucose, Fasting (BL3118)",
    "Abuse/Victimization (LLM)",
    "Halluc/Delusion/Paranoia (LLM)",
    "Lack of Support (LLM)",
    "Interpersonal Conflict (LLM)",
    # ... 필요한 변수들을 계속 추가하세요
]

# (C) 데이터 필터링 (지정한 변수만 남기기)
available_features = imp_pivot.index.tolist()

# 리스트 컴프리헨션은 원래 리스트(TARGET_FEATURES)의 순서를 보장합니다.
selected_features = [f for f in TARGET_FEATURES if f in available_features]
missing_features = set(TARGET_FEATURES) - set(selected_features)

if missing_features:
    print(
        f"\n[Warning] 다음 변수들은 데이터셋에 존재하지 않아 제외되었습니다:\n{missing_features}\n"
    )

if not selected_features:
    raise ValueError("선택된 변수가 하나도 없습니다. 변수 이름을 확인해주세요.")

# 선택된 변수 & 순서 그대로 데이터프레임 재정렬 (reindex 사용)
dir_top = dir_pivot.reindex(selected_features)
imp_top_filtered = imp_pivot.reindex(selected_features)

print(f"총 {len(dir_top)}개의 변수를 사용자가 지정한 순서대로 시각화합니다.")

# -----------------------------------------------------------------
# 4. (수정) 통합 플롯 그리기 (하나의 이미지로 저장)
# -----------------------------------------------------------------

print(f"총 {len(dir_top)}개의 변수를 하나의 히트맵으로 시각화합니다.")

# (A) 플롯 설정
# 변수 개수에 따라 높이 자동 조절 (변수당 0.5인치 + 기본 여백 3인치)
# 변수가 많아져도 셀이 찌그러지지 않도록 높이를 동적으로 할당합니다.
fig_height = len(dir_top) * 0.5 + 3
plt.figure(figsize=(12, fig_height))

font_sizes = {"title": 18, "label": 14, "tick": 12, "annot": 16, "colorbar_label": 14}

# 컬러바 설정
cbar_kws_dict = {
    "label": "Spearman Correlation (Feature Value vs. SHAP Value)",
    "shrink": 0.6,  # 전체 플롯 높이에 맞춰 컬러바 길이 비율 조정
    "ticklocation": "right",
}

# (B) 히트맵 그리기 (분할 없이 dir_top 전체 사용)
sns.heatmap(
    data=dir_top,  # ⬅️ 색상: 상관관계 (전체)
    annot=imp_top_filtered,  # ⬅️ 숫자: 중요도 (전체)
    fmt=".3f",
    cmap="coolwarm",
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=1.2,
    linecolor="gray",
    cbar_kws=cbar_kws_dict,
    annot_kws={"size": font_sizes["annot"], "weight": "bold", "color": "black"},
)

# (C) 레이블 및 타이틀 설정
plt.title(
    "SHAP Directionality (Selected Features)\n(Color=Direction, Text=Mean|SHAP|)",
    fontsize=font_sizes["title"],
    fontweight="bold",
    pad=25,
)
plt.xlabel("Time Point", fontsize=font_sizes["label"], fontweight="bold")
plt.ylabel("Feature", fontsize=font_sizes["label"], fontweight="bold")
plt.xticks(fontsize=font_sizes["tick"], rotation=0)
plt.yticks(fontsize=font_sizes["tick"])

# (D) 컬러바 폰트 및 위치 조정
cbar = plt.gca().collections[0].colorbar
cbar.set_label(
    cbar_kws_dict["label"], fontsize=font_sizes["colorbar_label"], fontweight="bold"
)
cbar.ax.tick_params(labelsize=font_sizes["tick"])

# (E) 저장
plt.tight_layout(rect=[0, 0, 1, 0.96])  # 타이틀 공간 확보
save_path = BASE_DIR / "Analysis1_SHAP_Heatmap_All_Selected.png"
plt.savefig(save_path, dpi=400)
plt.close()

print(f"저장 완료: {save_path}")

# # -----------------------------------------------------------------
# # 5. CSV 파일 저장 (Strongest, Consistency, Domain, Symbol + Importance 통합)
# # -----------------------------------------------------------------
# print("최종 상세 테이블 생성을 시작합니다...")


# # (A) 단순화된 기호 정의 (기준 0.3으로 상향)
# def get_corr_symbol_simple(val):
#     if pd.isna(val):
#         return ""
#     if val >= 0.3:
#         return "🔺"
#     if val <= -0.3:
#         return "▼"
#     return "• (Neutral)"


# dir_symbol_top = dir_top.applymap(get_corr_symbol_simple)
# # imp_top_filtered는 3번 섹션에서 이미 dir_top과 동일하게 정렬되었음

# # (B) 통합 테이블 생성
# combined_table = pd.DataFrame(index=dir_top.index)
# for tp in time_order:
#     if tp in dir_symbol_top.columns:
#         combined_table[f"{tp}_direction"] = dir_symbol_top[tp]
#         # [수정] 소수점 4자리 반올림
#         combined_table[f"{tp}_SHAP(mean)"] = imp_top_filtered[tp].round(4)

# # (C) Strongest window, Domain 추가
# strongest_window = imp_top_filtered.idxmax(axis=1)
# domains = combined_table.index.map(infer_domain)

# # (D) Top 20 Count (n/5) 계산 (컬럼명 변경)
# rank_pivot = imp_pivot.rank(axis=0, method="min", ascending=False)
# rank_pivot_top = rank_pivot.loc[dir_top.index]
# is_top_n = rank_pivot_top <= TOP_N
# consistency_count = is_top_n.sum(axis=1)
# consistency_str = consistency_count.astype(str) + f"/{len(time_order)}"

# # [수정] 컬럼 순서 변경
# combined_table.insert(0, "Domain", domains)
# combined_table["Strongest Window"] = strongest_window
# combined_table["Top 20 Count (n/5)"] = consistency_str

# # (F) 파일로 저장
# combined_table.to_excel(BASE_DIR / "short_fig/Analysis1_SHAP_Domain_Symbol_Importance_FINAL_Summary.xlsx")
# print(
#     f"★★★ 최종 상세 테이블 저장 완료: {BASE_DIR / 'short_fig/Analysis1_SHAP_Domain_Symbol_Importance_FINAL_Summary.xlsx'}"
# )


# # -----------------------------------------------------------------
# # 6. Top 20 도메인 비율 요약 테이블
# # -----------------------------------------------------------------
# print("\nTop 20 도메인 요약 테이블 생성을 시작합니다...")

# # (A) 각 시점별 Top 20 변수 추출 및 도메인 계산
# top_n_features_by_time = (
#     full_imp.groupby("time_point")
#     .apply(lambda x: x.nlargest(TOP_N, "mean_abs_shap"))
#     .reset_index(drop=True)
# )
# top_n_features_by_time["Domain"] = top_n_features_by_time["feature"].apply(infer_domain)

# # (B) 시점별 도메인 카운트 집계
# domain_counts_pivot = (
#     top_n_features_by_time.groupby("time_point")["Domain"]
#     .value_counts()
#     .unstack(level="time_point")
#     .fillna(0)
#     .astype(int)
# )
# domain_counts_pivot = domain_counts_pivot.reindex(columns=time_order, fill_value=0)

# # (C) 비율(%) 계산
# domain_proportions_pivot = domain_counts_pivot / TOP_N * 100

# # (D) 최종 포맷팅: "Proportion % (Count)"
# formatted_table = pd.DataFrame(
#     index=domain_counts_pivot.index, columns=domain_counts_pivot.columns
# )
# for col in formatted_table.columns:
#     counts = domain_counts_pivot[col]
#     props = domain_proportions_pivot[col]
#     formatted_table[col] = [
#         f"{p:.1f}% ({c})" if c > 0 else "-" for c, p in zip(counts, props)
#     ]

# # (E) 총합(Total) 행 추가
# total_counts = domain_counts_pivot.sum()
# total_row = [f"100.0% ({c})" for c in total_counts]
# formatted_table.loc["Total"] = total_row

# # (F) 파일로 저장
# domain_table_path = BASE_DIR / "Analysis1_SHAP_Domain_Proportions_Summary.xlsx"
# formatted_table.to_excel(domain_table_path)

# print(f"★★★ 신규 도메인 비율 요약 테이블 저장 완료: {domain_table_path}")
# print("\n모든 작업이 완료되었습니다.")